In [2]:
import tensorflow as tf

/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [3]:
print(tf.__version__)

2.0.0-beta1


In [4]:
from tensorflow.keras import Model, layers
from tensorflow.keras import backend as K
from tensorflow.python.keras.backend import _constant_to_tensor
from tensorflow.python.framework import ops
from tensorflow.python.ops import math_ops, clip_ops
import pdb
import numpy as np
import unittest

In [41]:
class LossFunction:
    
    @staticmethod
    def mean_squared_error(y_true, y_pred):
        '''
        loss = mean(square(y_true-y_pred))
        '''
        return tf.reduce_mean(tf.math.squared_difference(y_true, y_pred))
    
    @staticmethod
    def mean_absolute_error(y_true, y_pred):
        '''
        loss = abs(y_true-y_pred)
        '''
        return tf.reduce_mean(tf.math.abs(tf.math.subtract(y_true, y_pred)))
    
    @staticmethod
    def binary_crossentropy(y_true, y_pred, from_logits=False):
        def get_epsilon():
            # epsilon_value = 1e-7
            return tf.keras.backend.epsilon()
        
        if not from_logits:
#             if y_pred.op.type == "Sigmoid":
#                 TODO: dont clip. Use the from_logits directly.
            epsilon = get_epsilon()
            clipped_y_pred = tf.clip_by_value(y_pred, clip_value_min=epsilon, clip_value_max=(1.-epsilon))
            bce = tf.math.multiply(y_true, tf.math.log(tf.math.add(clipped_y_pred, epsilon)))
            temp = tf.math.multiply(tf.math.subtract(1., y_true), tf.math.log(tf.math.add(epsilon, tf.math.subtract(1., clipped_y_pred))))
            return tf.math.negative(tf.reduce_mean(tf.math.add(bce, temp)))
        else:
            # - x * z + log(1 + exp(x)), x = logits, z = labels
            return tf.reduce_mean(tf.math.add(tf.math.negative(tf.math.multiply(y_pred, y_true)), tf.math.log(tf.math.add(1., tf.math.exp(y_pred)))))
    
    @staticmethod
    def categorical_crossentropy(y_true, y_pred, from_logits=False):
        # Used when there are two or more label classes. Labels are expected to be provided in a one_hot representation. 
        # If the labels are to be provided as integers, SparseCategoricalCrossentropy loss is used. There should be #classes floating point values per feature.
        def get_epsilon():
            return tf.keras.backend.epsilon()
        if not from_logits:
            epsilon = get_epsilon()
            clipped_y_pred = tf.clip_by_value(y_pred, clip_value_min=epsilon, clip_value_max=(1.-epsilon))
            temp = math_ops.reduce_sum(y_true * math_ops.log(clipped_y_pred), axis=-1)
            return tf.math.negative(tf.reduce_mean(K.flatten(temp)))
    
    @staticmethod
    def hinge(y_true, y_pred):
        '''
        labels are expected to be -1 or 1.
        loss = max(1 - y_true * y_pred, 0)
        '''
        return tf.reduce_mean(tf.math.maximum(tf.subtract(1., tf.math.multiply(y_true, y_pred)), 0.))

    @staticmethod
    def kl_divergence(y_true, y_pred):
        '''
        loss = y_true * log(y_true / y_pred)
        '''
        return tf.reduce_sum(tf.math.multiply(y_true, tf.math.log(tf.math.divide(y_true, y_pred))))

In [42]:
class TestLossFunctions(unittest.TestCase):

    def test_mse(self):
        y_true = tf.Variable([0., 0., 1., 1.])
        y_pred = tf.Variable([1., 1., 1., 0.])
        x = tf.keras.backend.placeholder(shape=(None,))
        y = tf.keras.backend.placeholder(shape=(None,))

        mse = tf.keras.losses.MeanSquaredError()
        custom_mse = K.function([x,y], [LossFunction.mean_squared_error(x,y)]) 
        
        self.assertEqual(custom_mse([y_true, y_pred]), [mse(y_true, y_pred).numpy()])
    
    def test_mae(self):
        y_true = tf.Variable([0., 0., 1., 1.])
        y_pred = tf.Variable([1., 1., 1., 0.])
        x = tf.keras.backend.placeholder(shape=(None,))
        y = tf.keras.backend.placeholder(shape=(None,))
        
        mae = tf.keras.losses.MeanAbsoluteError()
        custom_mae = K.function([x,y], [LossFunction.mean_absolute_error(x,y)]) 
        
        self.assertEqual(custom_mae([y_true, y_pred]), [mae(y_true, y_pred).numpy()])
    
    def test_bce(self):
        y_true = tf.Variable([0., 0., 1., 1.])
        y_pred = tf.Variable([1., 1., 1., 0.])
        x = tf.keras.backend.placeholder(shape=(None,))
        y = tf.keras.backend.placeholder(shape=(None,))
        
        bce = tf.keras.losses.BinaryCrossentropy()
        custom_bce = K.function([x,y], [LossFunction.binary_crossentropy(x,y)]) 
        
        self.assertEqual(custom_bce([y_true, y_pred]), [bce(y_true, y_pred).numpy()])
    
    def test_cce(self):
        y_true = tf.Variable([[1., 0., 0.], [0., 1., 0.], [0., 0., 1.]])
        y_pred = tf.Variable([[.9, .05, .05], [.05, .89, .06], [.05, .01, .94]])
        x = tf.keras.backend.placeholder(shape=(None,))
        y = tf.keras.backend.placeholder(shape=(None,))
        
        cce = tf.keras.losses.CategoricalCrossentropy()
        custom_cce = K.function([x,y], [LossFunction.categorical_crossentropy(x,y)]) 
        
        self.assertEqual(custom_cce([y_true, y_pred]), [cce(y_true, y_pred).numpy()])
    
    def test_hinge(self):
        y_true = tf.Variable([-1., -1., 1., 1.])
        y_pred = tf.Variable([1., 1., 1., -1.])
        x = tf.keras.backend.placeholder(shape=(None,))
        y = tf.keras.backend.placeholder(shape=(None,))
        
        hinge = tf.keras.losses.Hinge()
        custom_hinge = K.function([x,y], [LossFunction.hinge(x,y)]) 
        
        self.assertEqual(custom_hinge([y_true, y_pred]), [hinge(y_true, y_pred).numpy()])
    
    def test_kld(self):
        y_true = tf.Variable([.4, .9, .2])
        y_pred = tf.Variable([.5, .8, .12])
        x = tf.keras.backend.placeholder(shape=(None,))
        y = tf.keras.backend.placeholder(shape=(None,))
        
        kld = tf.keras.losses.KLDivergence()
        custom_kld = K.function([x,y], [LossFunction.kl_divergence(x,y)]) 
        
        self.assertEqual(custom_kld([y_true, y_pred]), [kld(y_true, y_pred).numpy()])

In [43]:
unittest.main(argv=[''], verbosity=2, exit=False)

test_bce (__main__.TestLossFunctions) ... ok
test_cce (__main__.TestLossFunctions) ... ok
test_hinge (__main__.TestLossFunctions) ... ok
test_kld (__main__.TestLossFunctions) ... ok
test_mae (__main__.TestLossFunctions) ... ok
test_mse (__main__.TestLossFunctions) ... ok

----------------------------------------------------------------------
Ran 6 tests in 0.205s

OK
